# Part (c): Analysis Of Variance (ANOVA)
### UE22CS342AA2 - Data Analytics



- `Analysis of Variance (ANOVA)` is a hypothesis testing procedure used for comparing means from several groups simultaneously.
- Using an one-way ANOVA, we test whether the mean values of an outcome variable for different levels of a factor are different. Using multiple two sample t-tests to simultaneously test group means will result in incorrect estimation of the Type-I error; ANOVA overcomes this issue.
- In two-way ANOVA, we check the impact of more than one factor simultaneously on several groups.


## About the Dataset.

- abhicure.ai, a leading company in LLM development, has performed a study to assess employees' productivity score based on factors such as the work setting they practice.
- The work setting could be `Coworking Space` (Working on site, in a corporate environment), `Home office` (Work from home) or `Hybrid` and the communication tools they use could be `Zoom`, `Slack` or `Email`.
- As a data analyst for abhicure.ai, your job is to study the scenario and answer the following questions.
- Use the dataset `productivity_data_owa.csv` for tasks requiring only work setting and dataset `productivity_data_twa.csv` for tasks requiring both, work setting and communication tool.

### Problems
    - Problem 1
    - Problem 2
    - Problem 3
    - Problem 4
    - Problem 5


In [ ]:
# install necessary packages.
if (!requireNamespace("tidyverse", quietly = TRUE)) {
    install.packages("tidyverse")
}
if (!requireNamespace("moments", quietly = TRUE)) {
    install.packages("moments") 
}
if (!requireNamespace("readr", quietly = TRUE)) { 
    install.packages("readr")
}

In [ ]:
# load necessary packages
library(tidyverse)
library(moments) 
library(ggplot2)
library(readr)
library(dplyr)

In [ ]:
# load the dataset
data <- read.csv("/kaggle/input/anova-ws-c/productivity_data_owa.csv", header=TRUE)
head(data)

*Problem 1*

Does the data meet the normality assumption required for ANOVA? Verify for each group under the Work Setting using the Q-Q plot. Measure the skewness (if any) in each case. (2 points)
- As a bonus (not evaluated), do have a look at shapiro-wilk test from [here](https://www.geeksforgeeks.org/shapiro-wilk-test-in-r-programming/) to statistically infer normality! 

In [ ]:
# Your answer here.

In [ ]:
# Calculate skewness for each category in RemoteWorkSetting
skewness_results <- tapply(data$ProductivityScore, data$RemoteWorkSetting, skewness)

# Print the skewness results
print(skewness_results)


- We can see that the skewness of data under each category is small. The skewness is in the range (-1,1) hence they can be accepted as symmetrical.

In [ ]:
temp_data <- data.frame(
  RemoteWorkSetting = data$RemoteWorkSetting,
  ProductivityScore = data$ProductivityScore
)

In [ ]:
# Can verify through the bell curve as well.

In [ ]:

# Define a helper function to create bell curve plots
create_bell_curve_plot <- function(column_data, column_name) {
  # Get the mean of the column
  mu <- mean(column_data, na.rm = TRUE)
  # Get the standard deviation of the column
  sigma <- sd(column_data, na.rm = TRUE)
  # Creating a sequence of values within a range around the mean (mu) of the data
  # with 100 evenly spread out data points.
  x <- seq(mu - 3 * sigma, mu + 3 * sigma, length.out = 100)
  # Using the dnorm function to calculate the probability density values
  # of a normal distribution at specific x-values.
  y <- dnorm(x, mean = mu, sd = sigma)
  
  # Creating the ggplot object for the bell curve plot
  plot <- ggplot() +
    geom_line(aes(x, y), color = "black") +  
    geom_vline(xintercept = mu, color = "red", linetype = "dashed") +  
    geom_area(aes(x, y, fill = y), alpha = 0.5) +  
    scale_fill_gradient(low = "red", high = "black") +  
    labs(
      title = paste("PDF for", column_name),  
      x = "Productivity Score",  
      y = "Density"  
    ) +
    theme_minimal()  
  
  return(plot)
}

# Loop through each category in RemoteWorkSetting and create a PDF plot
unique_categories <- unique(data$RemoteWorkSetting)
for (category in unique_categories) {
  category_data <- data %>%
    filter(RemoteWorkSetting == category) %>%
    pull(ProductivityScore)
  
  plot <- create_bell_curve_plot(category_data, category)
  print(plot)
}

In [ ]:
for (category in unique_categories) {
  print(paste("Q-Q Plot for", category))
  
  # Extracting the ProductivityScore column as a numeric vector
  category_data <- data %>%
    filter(RemoteWorkSetting == category) %>%
    pull(ProductivityScore)
  
  print(length(category_data))
  
  # Create Q-Q plot
  qqnorm(category_data, main = paste("Q-Q Plot for", category))
# qqline(category_data)
}

- We observe that the skewness of each category under the work setting column is contained between (-1 and 1) hence, our assumption for normality holds good.
- Further, the Q-Q plot for each category is approximately in a straight line/ 45 degrees, hence the data is normal.

In [ ]:
# Shapiro Wilk test
for (category in unique_categories) {
  
  # Extracting the ProductivityScore column as a numeric vector
  category_data <- data %>%
    filter(RemoteWorkSetting == category) %>%
    pull(ProductivityScore)
  
  print(length(category_data))
  print(shapiro.test(category_data))
}

- All p values are more than 0.05 hence retain null hypotheses. Normally Distributed!

*Problem 2*

- One wants to determine if the work setting has any effect on productivity. How can this be inferred using statistical methods? Name the method and mathematically arrive at this inference. Can one use t-test for the same? Why/ Why not? *(NOTE: Assume the significance level to be 0.05)* (2 + 1 points)


In [ ]:
# Your answer here.

- The method used is One-way ANOVA.

In [ ]:
# Calculate the overall mean
overall_mean <- mean(data$ProductivityScore)

# Initialize sum of squares variables
ssb <- 0
sst <- 0
ssw <- 0

# Unique categories in RemoteWorkSetting
unique_categories <- unique(data$RemoteWorkSetting)

# Calculate category means and SSB
category_means <- sapply(unique_categories, function(category) {
  mean(data[data$RemoteWorkSetting == category, "ProductivityScore"])
})

# Calculate SSB, SSW, and SST
for (category in unique_categories) {
  category_data <- data[data$RemoteWorkSetting == category, "ProductivityScore"]
#     print(var(category_data))
  print(paste(category, mean(category_data)))
  n_category <- length(category_data)
  
  # Sum of Squares Between (SSB)
  ssb <- ssb + n_category * (category_means[category] - overall_mean)^2
  
  # Sum of Squares Within (SSW)
  ssw <- ssw + sum((category_data - category_means[category])^2)
}

# Total Sum of Squares (SST)
sst <- sum((data$ProductivityScore - overall_mean)^2)

# Degrees of freedom
df_between <- length(unique_categories) - 1
df_within <- length(data$ProductivityScore) - length(unique_categories)

# Mean Squares
msb <- ssb / df_between
msw <- ssw / df_within

# F-statistic
f_statistic <- msb / msw

# Print results
print(paste("SSB:", ssb, "MSB", msb))
print(paste("SSW:", ssw, "MSW", msw))
print(paste("SST:", sst))
print(paste("F-statistic:", f_statistic))


In [ ]:
p_value <- pf(f_statistic, df_between, df_within, lower.tail = FALSE)
# lower.tail is False because we are doing a right tailed test 
cat("p-value: ", p_value, "\n")

- Since the p-value is less than 0.05, we have sufficient evidence to conclude that there is statistically significant difference between the means of the three groups and hence work setting has an effect on productivity.
- t-test is used to compare the means of 2 groups. If more than 2 groups exist, using t-test will yield incorrect Type-1 and Type-2 errors, hence t-test must be avoided in favor of ANOVA. 

**Important**
*To check for homogenity of variance, do have look at Levene's test*

*Levene's* test prooves the homogenity of variance as we retain the levene null hypothesis.

In [ ]:
library(car)
result = leveneTest(ProductivityScore ~ RemoteWorkSetting, data)
result

## Post-hoc tests.
- When you use ANOVA to test the equality of at least three group means, statistically significant results indicate that not all of the group means are equal. However, ANOVA results do not identify which particular differences between pairs of means are significant. 
- In such a case, we use post-hoc tests, also known as multiple comparisons.
- There are a variety of post hoc tests you can choose from, but `Tukey’s method` is the most common for comparing all possible group pairings.
- If the adjusted p-value between a pair of groups is less than 0.05, then we can assume statistical significance between the pair.
- You can learn more [here](https://statisticsbyjim.com/anova/post-hoc-tests-anova/).

*Problem 3*

Perform Tukey's Test to infer which pairs have means statistically significant from one other and which dont. (State your answer for significance level at 0.05 and 0.02). Which pair of categories show a *clear* difference in productivity?  (1 + 1 points)

In [ ]:
# Your answer here.

In [ ]:
# Answer.
fit <- aov(ProductivityScore ~ RemoteWorkSetting, data = data)
anova_result <- summary(fit)

# Perform Tukey's HSD test
# HSD stands for honest significance difference.
tukey_result <- TukeyHSD(fit)

# Print Tukey's HSD results
print(tukey_result)

# Plot Tukey's HSD results (optional)
plot(tukey_result)

- For both, 0.05 and at 0.02, we can conclude that only `Home office-Coworking space` and `Hybrid-Home office` have means that are of statistical significance.No sufficient evidence to prove insignificance between the pair, Hybrid-Coworking space. (If the significance level was more lenient, we might have been able to.)
- Hence both `Home office-Coworking space` and `Hybrid-Home office` show clear difference.

# Two-way ANOVA

- Two-way ANOVA is used when the response variable is influenced by several factors.
- Consider the `productivity_twa.csv`, where in now the productivity of the employees is influenced by both, the `work setting` and the `communication tool`. 
- The work setting could be `Coworking Space` (Working on site, in a corporate environment), `Home office` (Work from home) or `Hybrid` and the communication tools they use could be `Zoom`, `Slack` or `Email`.

*Problem 4*

Does the interaction between different work settings and communication tools significantly affect employee productivity scores? Justify your answer. (2 points)

In [ ]:
# your answer here

In [ ]:
# load the libraries necessary.
library(ggplot2) 
library(readr) 
library(stats)

In [ ]:
# loading the dataset
data <- read.csv("/kaggle/input/anova-ws-c/productivity_data_twa.csv", header=TRUE)
head(data)

In [ ]:
# getting the statistics of 2 way anova
anova_result <- aov(ProductivityScore ~ RemoteWorkSetting + CommunicationTool + RemoteWorkSetting:CommunicationTool, data = data)
summary(anova_result)

- The observed p-value of the combination of remote work setting and communication tool is 0.1454 (>0.05) hence we dont have sufficient evidence to reject the null hypothesis and conclude that the interaction between different work settings and communication tools has no significant effect on employee productivity scores
- `***`: p-value ≤ 0.001 (Highly significant)
- `**` : 0.001 < p-value ≤ 0.01 (Very significant)
- `*` : 0.01 < p-value ≤ 0.05 (Significant)
- `.` : 0.05 < p-value ≤ 0.1 (Marginally significant)
- `' '` : p-value > 0.1 (Not significant)

*Levene's Test for Homogenity of variance*

In [ ]:
result = leveneTest(ProductivityScore ~ interaction(RemoteWorkSetting, CommunicationTool), 
                    data = data)
 
# print the result
print(result)

We observe that since the p-value is high, we retain the null hypothesis - The variance is homogeneous.

In [ ]:
# Extract residuals from the ANOVA model
residuals <- residuals(anova_result)

# Perform Shapiro-Wilk test for normality on the residuals
shapiro_test <- shapiro.test(residuals)

# Print the result of the Shapiro-Wilk test
print(shapiro_test)

*Problem 5*

Having learnt from the above study, which pair of work setting and communication tool must be adopted by the management for the employees? Based on the study, would you, as an entrepreneur adopt the same strategy for your company? Justify your answer. What would the name of your company be? 👀 (1 point)


In [ ]:
# your answer here.

In [ ]:
# group based on pairs of remote work setting and communication setting in descending order.
grouped_data <- data %>%
  group_by(RemoteWorkSetting, CommunicationTool) %>%
  summarise(
    count = n(),
    mean = mean(ProductivityScore, na.rm = TRUE),
    std = sd(ProductivityScore, na.rm = TRUE),
    min = min(ProductivityScore, na.rm = TRUE),
    q25 = quantile(ProductivityScore, 0.25, na.rm = TRUE),
    median = median(ProductivityScore, na.rm = TRUE),
    q75 = quantile(ProductivityScore, 0.75, na.rm = TRUE),
    max = max(ProductivityScore, na.rm = TRUE)
  ) %>%
  arrange(desc(mean))

# Print the grouped data
print(grouped_data)

- From the above data, we can see that the combination of `coworking space` and `email` will guide the employees of abhicure.ai to maximum productivity.
- The above results were from a study by and for abhicure.ai. The factors at every company can vary based on several factors such as place, employees, etc. Thus I would not adopt the strategy that did the best for the employees of abhicure.ai.

*fin*